In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('instance').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/02 16:42:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/02 16:42:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark.createDataFrame({"season": ["winter","summer"], "wind_speed_ms": [4.5, 7.5]})

PySparkTypeError: [CANNOT_INFER_SCHEMA_FOR_TYPE] Can not infer schema for type: `str`.

24/07/01 01:11:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [11]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

def plus_one(v):
    return v + 1

plus_one_udf = F.udf(plus_one, T.IntegerType())

plus_one_udf = spark.udf.register('plus_one_udf', plus_one, T.IntegerType())

plus_one_udf(F.lit(5))

24/07/02 16:46:18 WARN SimpleFunctionRegistry: The function plus_one_udf replaced a previously registered function.


Column<'plus_one_udf(5)'>

In [41]:
import pyspark.sql.functions as F
df = spark.createDataFrame([(1, 3, 4, 25, 1, None,),
                            (2, 6, 7, 2, 2, None,),
                            (3, 3, None, 25, 3, None,),
                            (4, None, None, 3, 2, None),
                            (5, None, None, None, 2, None),
                            (6, 3, 2, 25, 2, None),]
                          , schema='transactionId int, predError int, value int, storeId int, productId int, f int')
df.filter(F.col("storeId")==25).show()
df.filter(F.col("storeId")==25).select("predError","storeId").show()
df.filter(F.col("storeId")==25).select("predError","storeId").distinct().show()

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            1|        3|    4|     25|        1|NULL|
|            3|        3| NULL|     25|        3|NULL|
|            6|        3|    2|     25|        2|NULL|
+-------------+---------+-----+-------+---------+----+

+---------+-------+
|predError|storeId|
+---------+-------+
|        3|     25|
|        3|     25|
|        3|     25|
+---------+-------+

+---------+-------+
|predError|storeId|
+---------+-------+
|        3|     25|
+---------+-------+



In [37]:
df.select('transactionId', 'predError', 'value', 'f').show()
df.select(['transactionId', 'predError', 'value', 'f']).show()
df.select(*['transactionId', 'predError', 'value', 'f']).show()

+-------------+---------+-----+----+
|transactionId|predError|value|   f|
+-------------+---------+-----+----+
|            1|        3|    4|NULL|
|            2|        6|    7|NULL|
|            3|        3| NULL|NULL|
|            4|     NULL| NULL|NULL|
|            5|     NULL| NULL|NULL|
|            6|        3|    2|NULL|
+-------------+---------+-----+----+

+-------------+---------+-----+----+
|transactionId|predError|value|   f|
+-------------+---------+-----+----+
|            1|        3|    4|NULL|
|            2|        6|    7|NULL|
|            3|        3| NULL|NULL|
|            4|     NULL| NULL|NULL|
|            5|     NULL| NULL|NULL|
|            6|        3|    2|NULL|
+-------------+---------+-----+----+

+-------------+---------+-----+----+
|transactionId|predError|value|   f|
+-------------+---------+-----+----+
|            1|        3|    4|NULL|
|            2|        6|    7|NULL|
|            3|        3| NULL|NULL|
|            4|     NULL| NULL|NULL|

In [23]:
type(df), type(df.write)

(pyspark.sql.dataframe.DataFrame, pyspark.sql.readwriter.DataFrameWriter)

In [51]:
df.filter(F.col('value').isNotNull()).show()
df.filter(~F.isnull('value')).show()

df.filter(F.col('predError').isin([3, 6])).show()
df.filter(~F.col('predError').isin([3, 6])).show()

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            1|        3|    4|     25|        1|NULL|
|            2|        6|    7|      2|        2|NULL|
|            6|        3|    2|     25|        2|NULL|
+-------------+---------+-----+-------+---------+----+

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            1|        3|    4|     25|        1|NULL|
|            2|        6|    7|      2|        2|NULL|
|            6|        3|    2|     25|        2|NULL|
+-------------+---------+-----+-------+---------+----+

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            1|        3|    4|     25|        1|NULL|
|       

In [56]:
df.drop('value').show()
df.drop('value', 'f').show()
df.drop(['value', 'f']).show()

+-------------+---------+-------+---------+----+
|transactionId|predError|storeId|productId|   f|
+-------------+---------+-------+---------+----+
|            1|        3|     25|        1|NULL|
|            2|        6|      2|        2|NULL|
|            3|        3|     25|        3|NULL|
|            4|     NULL|      3|        2|NULL|
|            5|     NULL|   NULL|        2|NULL|
|            6|        3|     25|        2|NULL|
+-------------+---------+-------+---------+----+

+-------------+---------+-------+---------+
|transactionId|predError|storeId|productId|
+-------------+---------+-------+---------+
|            1|        3|     25|        1|
|            2|        6|      2|        2|
|            3|        3|     25|        3|
|            4|     NULL|      3|        2|
|            5|     NULL|   NULL|        2|
|            6|        3|     25|        2|
+-------------+---------+-------+---------+



PySparkTypeError: [NOT_COLUMN_OR_STR] Argument `col` should be a Column or str, got list.